# Create materialized lake views 
1. Use this notebook to create materialized lake views. 
2. Select **Run all** to run the notebook. 
3. When the notebook run is completed, return to your lakehouse and refresh your materialized lake views graph. 


In [ ]:
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;

In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS silver.customer;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS silver.customer
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
WITH cte_countries AS (
SELECT 
    DISTINCT country_region_code,
    territory_group,
    territory_name,
    CASE 
        WHEN country_region_code IN ('AU', 'DE', 'ENG', 'FR', 'GB') THEN territory_name
        WHEN country_region_code IN ('17', '31', '41', '45','57', '59', '62', '75', '77', 
            '78', '80', '91', '92', '93', '94', '95', 'BC', 'NSW', 'QLD', 'VIC' ) THEN territory_group
        WHEN country_region_code = 'CA' AND territory_name = 'Canada' THEN 'Canada'
        WHEN country_region_code = 'CA' AND territory_name = 'California' THEN 'United States of America'
        WHEN country_region_code IN ('US', 'WA') THEN 'United States of America'
    ELSE 'Undefined'
    END AS country
FROM bronze.customer 
)
SELECT 
    DISTINCT cu.customer_id,
    CONCAT_WS(' ', cu.first_name, cu.middle_name, cu.last_name) AS customer_name,
    CONCAT_WS(' ', cu.address_line_1, cu.addess_line_2) AS address,
    cu.city,
    cu.state_name AS state,
    ct.country,
    CONCAT_WS(
        ', ', 
        CONCAT_WS(' ', cu.address_line_1, cu.addess_line_2), 
        cu.city, 
        cu.state_name, 
        ct.country
    ) AS full_address,
    cu.start_date,
    cu.end_date,
    cu.is_active
FROM bronze.customer cu
JOIN cte_countries ct
    ON ct.country_region_code = cu.country_region_code AND 
       ct.territory_group     = cu.territory_group     AND
       ct.territory_name      = cu.territory_name

In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS silver.product;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS silver.product
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
SELECT 
    p.product_id, 
    p.product,
    s.product_subcategory,
    c.product_category,
    p.color,
    p.size,
    p.product_line,
    p.class,
    p.style,
    p.product_model_id
FROM bronze.product p
JOIN bronze.product_subcategory s
    ON s.product_subcategory_id = p.product_subcategory_id
JOIN bronze.product_category c
    ON c.product_category_id = s.product_category_id

In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS silver.sales;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS silver.sales
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
SELECT
    d.sales_order_id,
    d.sales_order_detail_id,
    d.sales_detail_item,
    d.items_per_order,
    h.order_date,
    h.ship_date,
    h.due_date,
    h.customer_id, 
    d.product_id,
    d.order_qty,
    d.unit_price,
    d.percentage_discount,
    c.standard_cost AS unit_cost
FROM bronze.sales_order_detail d
JOIN bronze.sales_order_header h
    ON h.sales_order_id = d.sales_order_id
JOIN bronze.product_cost_history c
    ON c.product_id = d.product_id AND
       h.order_date >= c.start_date AND
       (h.order_date <= c.end_date OR c.is_active = 1)


In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS silver.tax_and_freight;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS silver.tax_and_freight
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
SELECT
    *
FROM bronze.sales_order_header

In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS gold.fact_tax_and_freight;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS gold.fact_tax_and_freight
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
SELECT * FROM silver.tax_and_freight

In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS gold.fact_sales;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS gold.fact_sales
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
SELECT * FROM silver.sales

In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS gold.dim_customer;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS gold.dim_customer
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
SELECT * FROM silver.customer

In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS gold.dim_product;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS gold.dim_product
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
SELECT * FROM silver.product

In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS gold.aux_sales_order;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS gold.aux_sales_order
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
SELECT DISTINCT sales_order_id FROM silver.sales

In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS gold.dim_date;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS gold.dim_date
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
SELECT
  d as date,
  year(d)  AS year,
  month(d) AS month,
  element_at(
    array('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'),
    month(d)
  ) AS month_name,
  day(d)   AS day
FROM (
  SELECT explode(
    sequence(
      to_date('2022-01-01'),
      to_date(concat(year(current_date()), '-12-31')),
      interval 1 day
    )
  ) AS d
) t
ORDER BY d;


In [ ]:
DROP MATERIALIZED LAKE VIEW IF EXISTS gold.agg_sales;

CREATE MATERIALIZED LAKE VIEW IF NOT EXISTS gold.agg_sales
TBLPROPERTIES (delta.enableChangeDataFeed = true)
AS
WITH base AS (
SELECT 
    date_trunc('month', order_date) AS month,
    SUM(order_qty * unit_price) AS revenue,
    SUM(order_qty * unit_price * percentage_discount) AS discount,
    SUM(order_qty * unit_cost) AS total_cost
FROM gold.fact_sales
GROUP BY date_trunc('month', order_date)
)
SELECT 
    b.*,
    revenue - discount AS sales_amount,
    revenue - discount - total_cost AS margin,
    (revenue - discount - total_cost) / total_cost AS percentage_margin
FROM base b
ORDER BY month;